# Method 1

In [3]:
import pandas as pd
from fuzzywuzzy import process
import os

base_dir = "/Users/zac/Codes/Music_Project/GIT_HUB/Musis_Recommendation_Engine/data/kaggle_dataset/combined.csv"

# Load the CSV files
sheet1 = pd.read_csv('/Users/zac/Codes/Music_Project/GIT_HUB/Musis_Recommendation_Engine/data/kaggle_dataset/combined.csv')
sheet2 = pd.read_csv('/Users/zac/Codes/Music_Project/GIT_HUB/Musis_Recommendation_Engine/data/processed_data/crowd_sourced_data/MusicMood_Survey_Final_new.csv', encoding='ISO-8859-1')

sheet2 = sheet2.rename(columns={"ARTIST":"artist_name", "NAME":"track_name", "MOOD":"mood"})


In [ ]:

# Create a dictionary for quick lookups
mood_dict = {}
for _, row in sheet2.iterrows():
    key = (row['artist_name'].strip().lower(), row['track_name'].strip().lower())
    mood_dict[key] = row['mood']

# Function to find the best match for track_name and artist_name
def find_mood(artist_name, track_name, mood_dict):
    key = (artist_name.strip().lower(), track_name.strip().lower())
    if key in mood_dict:
        return mood_dict[key]
    
    # If exact match not found, perform fuzzy matching
    keys = list(mood_dict.keys())
    best_match_artist, artist_score = process.extractOne(artist_name, [k[0] for k in keys])
    best_match_track, track_score = process.extractOne(track_name, [k[1] for k in keys])
    
    if artist_score > 80 and track_score > 80:  # Adjust threshold as needed
        for k in keys:
            if k[0] == best_match_artist and k[1] == best_match_track:
                return mood_dict[k]
    
    return None

# Add the mood column to sheet1
sheet1['mood'] = sheet1.apply(lambda row: find_mood(row['artist_name'], row['track_name'], mood_dict), axis=1)

# Save the result to a new CSV file
sheet1.to_csv('sheet1_with_mood.csv', index=False)


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz, process
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
import numpy as np

# # Load the sheets
# sheet1 = pd.read_csv('sheet1.csv')
# sheet2 = pd.read_csv('sheet2.csv')

# Function to match songs with fuzzy matching
def match_mood(row):
    track_name = row['track_name']
    artist_name = row['artist_name']
    query = f"{track_name} {artist_name}"
    choices = sheet2.apply(lambda x: f"{x['track_name']} {x['artist_name']}", axis=1).tolist()
    best_match, score = process.extractOne(query, choices, scorer=fuzz.token_sort_ratio)
    if score >= 80:  # Adjust the threshold as needed
        matched_row = sheet2.iloc[choices.index(best_match)]
        return matched_row['mood']
    return None

# Apply fuzzy matching with multiprocessing
def parallel_apply(df, func, num_processes=cpu_count()):
    df_split = np.array_split(df, num_processes)
    pool = Pool(num_processes)
    results = pool.map(func, df_split)
    pool.close()
    pool.join()
    return pd.concat(results)

# Function to apply fuzzy matching with progress bar
def apply_fuzzy_matching(df):
    tqdm.pandas()
    df['mood'] = df.progress_apply(match_mood, axis=1)
    return df

# Apply fuzzy matching to sheet1
sheet1 = parallel_apply(sheet1, apply_fuzzy_matching)

# Save the result to a new CSV file
sheet1.to_csv('sheet1_with_mood_parranew.csv', index=False)


In [6]:
import pandas as pd
from fuzzywuzzy import fuzz, process
from tqdm.notebook import tqdm

# # Load the CSV files
# sheet1 = pd.read_csv('sheet1.csv')
# sheet2 = pd.read_csv('sheet2.csv')

# Cache the choices for efficiency
def get_choices(df):
    return df.apply(lambda x: f"{x['track_name']} {x['artist_name']}", axis=1).tolist()

choices = get_choices(sheet2)

# Function to match songs with fuzzy matching considering both track name and artist name
def get_best_match(row, choices):
    query = f"{row['track_name']} {row['artist_name']}"
    best_match, score = process.extractOne(query, choices, scorer=fuzz.token_sort_ratio)
    if score >= 80:  # Adjust the threshold as needed
        return best_match
    return None

# Add a new column for mood
sheet1['mood'] = None

# Perform fuzzy matching with a progress bar
for idx, row in tqdm(sheet1.iterrows(), total=sheet1.shape[0], desc="Matching moods"):
    best_match = get_best_match(row, choices)
    if best_match:
        matched_row = sheet2.loc[choices.index(best_match)]
        sheet1.at[idx, 'mood'] = matched_row['mood']

# Save the result to a new CSV file
sheet1.to_csv('sheet1_with_mood.csv', index=False)


Matching moods:   0%|          | 0/1331673 [00:00<?, ?it/s]